In [4]:
import pandas as pd

In [6]:
df = pd.read_csv('venues_all.csv')
df.head()

,name,categories,lat,lng
0,Sayaji,Hotel,18.599535,73.754995
1,Natural Ice Cream,Ice Cream Shop,18.591192,73.752440
2,Barbeque Nation,BBQ Joint,18.599390,73.755090
3,Courtyard by Marriott,Hotel,18.591527,73.746831
4,Little Italy,Italian Restaurant,18.591513,73.743668


In [7]:
df.shape

(424, 4)

In [8]:
from sklearn.cluster import DBSCAN

In [13]:
Clus_dataSet = df[['lat','lng']] 
Clus_dataSet.head()

,lat,lng
0,18.599535,73.754995
1,18.591192,73.752440
2,18.599390,73.755090
3,18.591527,73.746831
4,18.591513,73.743668


In [26]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [28]:
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)
Clus_dataSet[0:5]

array([[ 0.86033558, -1.25272439],
       [ 0.66849366, -1.30300637],
       [ 0.85698296, -1.25085174],
       [ 0.67620385, -1.41335426],
       [ 0.67587536, -1.47558923]])

In [95]:
db = DBSCAN(eps=0.2, min_samples=5).fit(Clus_dataSet)
db

DBSCAN(algorithm='auto', eps=0.2, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=5, n_jobs=None, p=None)

In [96]:
labels = db.labels_
labels[:250]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  2,  0,  2,  0,  1,
        3,  0,  1,  1,  0,  1,  1,  4,  3,  0,  0,  4,  0,  2,  2,  2,  0,
        1,  0,  2,  2,  0,  1,  2,  0,  0,  0,  0,  0,  4,  1,  0,  2,  1,
        0,  2,  1,  0,  2,  0,  0,  1,  1,  0,  1,  2,  1,  2,  3,  3,  0,
       -1,  4,  2,  0,  0,  1,  1,  2, -1,  0, -1,  1,  1,  0,  0,  2,  2,
        0, -1,  0,  0, -1,  1,  1,  4,  3,  1,  2,  2, -1,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  4,  5,  4,  2,  5,  4,  5,  5,  5,  5,  5,  2,
        4, -1,  5,  4,  4,  4,  5,  5,  4, -1,  4,  4, -1, -1,  5,  4,  4,
       -1,  5,  4,  5,  5,  5,  5,  4,  4,  4,  2, -1,  2, -1, -1,  5,  2,
        2,  4,  2,  2, -1,  2,  4,  2,  2, -1,  2,  2,  0,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  6,  1,  1,  1,  6,  1,  6,  6,  1,  6,  6,  6,  1,  6,
        1,  6,  1,  6,  1,  6,  6,  1,  6,  1,  6,  6,  6,  6,  6,  6,  6,
        6, -1, -1,  1,  1

In [97]:
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters

9

In [98]:
print(metrics.silhouette_score(Clus_dataSet, labels))


0.4720126553195963


In [99]:
import folium
from folium import plugins

In [100]:
color_options = ['black', 'blue', 'cadetblue', 'darkblue', 'darkgreen', 'darkpurple', 'darkred', 'gray', 'green', 'lightblue', 'lightgreen', 'lightred', 'orange', 'pink', 'purple', 'red']

In [101]:
map_ = folium.Map(location=[df.lat[0], df.lng[0]], zoom_start=11)
# add markers to map
for lat, lng, Categories, name, lab in zip(df['lat'], df['lng'], df['categories'], df['name'], labels):
    label = '{}, {}'.format(name, Categories, lab)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color=color_options[lab],
        fill=True,
        fill_opacity=1,
        parse_html=False).add_to(map_)  
    
map_